In [13]:
import pandas as pd

# 1. Get data

## 1.1. Dades obertes catalunya

In [4]:
# df = pd.read_csv('../../dades/font/exemple.csv')
df_csv = pd.read_csv('https://github.com/fndvit/mds-udg-viz-i-dgpcd-hackathon-25/raw/refs/heads/main/src/dades/font/certificats.csv')

/var/folders/0b/9qg1xh5d46j8phx5051jbncw0000gn/T/ipykernel_61565/2859823496.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_csv = pd.read_csv('https://github.com/fndvit/mds-udg-viz-i-dgpcd-hackathon-25/raw/refs/heads/main/src/dades/font/certificats.csv')


## 1.2. Dades INE

In [28]:
urls_ine = {
    'renta_media_mediana': {
        'key': 'Indicadores de renta media y mediana',
        'urls': [
            "https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/31016.csv?nocab=1",
            "https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/30896.csv?nocab=1",
            "https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/31223.csv?nocab=1",
            "https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/31079.csv?nocab=1"
        ]
    },
    'distr_fuente_ingresos': {
        'key': 'Distribución por fuente de ingresos',
        'urls': [
            "https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/31017.csv?nocab=1",
            "https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/30897.csv?nocab=1",
            "https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/31224.csv?nocab=1",
            "https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/31080.csv?nocab=1"
        ],
    },
    'indice_gini': {
        'key': 'Índice de Gini y Distribución de la renta P80/P20',
        'urls': [
            "https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/37697.csv?nocab=1",
            "https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/37686.csv?nocab=1",
            "https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/37718.csv?nocab=1",
            "https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/37704.csv?nocab=1",
        ],
    },
    'demographics': {
        'key': 'Indicadores demográficos',
        'urls': [
            "https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/31024.csv?nocab=1",
            "https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/30904.csv?nocab=1",
            "https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/31231.csv?nocab=1",
            "https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/31087.csv?nocab=1",
        ]
    },
}

In [29]:
# Montem un objecte 
# codi_districte: {
#       codi_districte,
#       codi_postal,
#       poblacio,
#       years: {
#           2020: {
#               // excel renta media per persona
#               renta_neta_mitja_por_persona,
#               renta_neta_mitja_por_hogar,
#               renta_mitja_por_unitat_consum,
#               renta_mediana_por_unitat_consum,
#               renta_bruta_mitja_por_persona,
#               renta_bruta_mitja_por_hogar,
#               
#               // excel fuente ingresos
#               renta_bruta_mitja_por_persona_fuente_salario,
#               renta_bruta_mitja_por_persona_fuente_pensiones,
#               renta_bruta_mitja_por_persona_fuente_desempleo,
#               renta_bruta_mitja_por_persona_fuente_otras_prestaciones,
#               renta_bruta_mitja_por_persona_fuente_otros_ingresos,
#
#               // excel gini
#               indice_gini,
#               distribucion_renta_p80_p20,
#               
#               // excel demographics
#               edad_media_poblacion,
#               prcnt_poblacion_menor_edad,
#               prcnt_poblacion_mas_65_anos,
#               tamano_medio_hogar,
#               prcnt_hogares_unipersonales,
#               poblacion_habitantes,
#               prcnt_poblacion_espanola,
#           }
#       }
# }

In [30]:
def initRow(res, row): 
    seccio = row['Secciones'].split()[0]
    # Validem estructura
    if not seccio in res:
        municipios = row.get('Municipios', None)  # Evita KeyError
        
        if municipios:
            municipios = municipios.split()
            codi_postal = municipios[0] if len(municipios) > 0 else ""
            poblacio = municipios[1] if len(municipios) > 1 else ""
        else:
            codi_postal = ""
            poblacio = ""

        res[seccio] = {
            'codi_districte': seccio,
            'codi_postal': codi_postal,
            'poblacio': poblacio,
            'years': {}
        }

    
    if not row['Periodo'] in res[seccio]['years']:
        res[seccio]['years'][row['Periodo']] = {
            'renta_neta_mitja_por_persona': None,
            'renta_neta_mitja_por_hogar': None,
            'renta_mitja_por_unitat_consum': None,
            'renta_mediana_por_unitat_consum': None,
            'renta_bruta_mitja_por_persona': None,
            'renta_bruta_mitja_por_hogar': None,
            'renta_bruta_mitja_por_persona_fuente_salario': None,
            'renta_bruta_mitja_por_persona_fuente_pensiones': None,
            'renta_bruta_mitja_por_persona_fuente_desempleo': None,
            'renta_bruta_mitja_por_persona_fuente_otras_prestaciones': None,
            'renta_bruta_mitja_por_persona_fuente_otros_ingresos': None,
            'indice_gini': None,
            'distribucion_renta_p80_p20': None,
            'edad_media_poblacion': None,
            'prcnt_poblacion_menor_edad': None,
            'prcnt_poblacion_mas_65_anos': None,
            'tamano_medio_hogar': None,
            'prcnt_hogares_unipersonales': None,
            'poblacion_habitantes': None,
            'prcnt_poblacion_espanola': None,
        }



In [31]:
res = {}

In [32]:
map_excel_categoria = {
    "Renta neta media por persona": "renta_neta_mitja_por_persona",
    "Renta neta media por hogar": "renta_neta_mitja_por_hogar",
    "Media de la renta por unidad de consumo": "renta_mitja_por_unitat_consum",
    "Mediana de la renta por unidad de consumo": "renta_mediana_por_unitat_consum",
    "Renta bruta media por persona": "renta_bruta_mitja_por_persona",
    "Renta bruta media por hogar": "renta_bruta_mitja_por_hogar",
    "Fuente de ingreso: salario": "renta_bruta_mitja_por_persona_fuente_salario",
    "Fuente de ingreso: pensiones": "renta_bruta_mitja_por_persona_fuente_pensiones",
    "Fuente de ingreso: prestaciones por desempleo": "renta_bruta_mitja_por_persona_fuente_desempleo",
    "Fuente de ingreso: otras prestaciones": "renta_bruta_mitja_por_persona_fuente_otras_prestaciones",
    "Fuente de ingreso: otros ingresos": "renta_bruta_mitja_por_persona_fuente_otros_ingresos",
    "Índice de Gini": "indice_gini",
    "Distribución de la renta P80/P20": "distribucion_renta_p80_p20",
    "Edad media de la población": "edad_media_poblacion",
    "Porcentaje de población menor de 18 años": "prcnt_poblacion_menor_edad",
    "Porcentaje de población de 65 y más años": "prcnt_poblacion_mas_65_anos",
    "Tamaño medio del hogar": "tamano_medio_hogar",
    "Porcentaje de hogares unipersonales": "prcnt_hogares_unipersonales",
    "Población": "poblacion_habitantes",
    "Porcentaje de población española": "prcnt_poblacion_espanola",
}

In [51]:
for type in urls_ine.keys():
    print(f"Processing {type}")
    for url in urls_ine[type]['urls']:
        print(f"- Processing {url}")
        temp_df = pd.read_csv(url, sep=";", encoding="utf-8-sig", decimal=",")

        for index, row in temp_df.iterrows():
            # Només els que tinguin totes les dades
            if pd.notna(row['Secciones']) and row['Secciones'] != '' and row['Secciones'] != None:
                seccio = row['Secciones'].split()[0]
                initRow(res, row)

                # Processem les dades
                res[seccio]['years'][row['Periodo']][map_excel_categoria[row[urls_ine[type]['key']]]] = (
                    row['Total'] if isinstance(row['Total'], float) 
                    else pd.to_numeric(str(row['Total']).replace(".", "").replace(",", "."), errors='coerce')
)


Processing renta_media_mediana
- Processing https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/31016.csv?nocab=1
- Processing https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/30896.csv?nocab=1
- Processing https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/31223.csv?nocab=1
- Processing https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/31079.csv?nocab=1
Processing distr_fuente_ingresos
- Processing https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/31017.csv?nocab=1
- Processing https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/30897.csv?nocab=1
- Processing https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/31224.csv?nocab=1
- Processing https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/31080.csv?nocab=1
Processing indice_gini
- Processing https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/37697.csv?nocab=1
- Processing https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/37686.csv?nocab=1
- Processing https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/37718.csv?nocab=1
- Processing https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/37704.c

Anem a preparar les dades per montar un dataset

In [52]:
res_res = []
for codi_districte in res.keys():
    for year in res[codi_districte]['years'].keys():
        res_res.append({
            **{ 'codi_districte': codi_districte, 'codi_postal': res[codi_districte]['codi_postal'], 'poblacio': res[codi_districte]['poblacio'], 'year': year },
            **res[codi_districte]['years'][year],
        })

In [53]:
res_res

[{'codi_districte': '1700101001',
  'codi_postal': '17001',
  'poblacio': 'Agullana',
  'year': 2022,
  'renta_neta_mitja_por_persona': 19392,
  'renta_neta_mitja_por_hogar': 50407,
  'renta_mitja_por_unitat_consum': 28845,
  'renta_mediana_por_unitat_consum': 19950,
  'renta_bruta_mitja_por_persona': 27842.0,
  'renta_bruta_mitja_por_hogar': 72374,
  'renta_bruta_mitja_por_persona_fuente_salario': 11573.0,
  'renta_bruta_mitja_por_persona_fuente_pensiones': 3936.0,
  'renta_bruta_mitja_por_persona_fuente_desempleo': 221.0,
  'renta_bruta_mitja_por_persona_fuente_otras_prestaciones': 424.0,
  'renta_bruta_mitja_por_persona_fuente_otros_ingresos': 11688.0,
  'indice_gini': 30.8,
  'distribucion_renta_p80_p20': 2.7,
  'edad_media_poblacion': 44.0,
  'prcnt_poblacion_menor_edad': 19.1,
  'prcnt_poblacion_mas_65_anos': 20.6,
  'tamano_medio_hogar': 2.6,
  'prcnt_hogares_unipersonales': 26.3,
  'poblacion_habitantes': 921,
  'prcnt_poblacion_espanola': 83.8},
 {'codi_districte': '1700101001

In [54]:
df = pd.DataFrame(res_res)

In [56]:
df

,codi_districte,codi_postal,poblacio,year,renta_neta_mitja_por_persona,renta_neta_mitja_por_hogar,renta_mitja_por_unitat_consum,renta_mediana_por_unitat_consum,renta_bruta_mitja_por_persona,renta_bruta_mitja_por_hogar,...,renta_bruta_mitja_por_persona_fuente_otros_ingresos,indice_gini,distribucion_renta_p80_p20,edad_media_poblacion,prcnt_poblacion_menor_edad,prcnt_poblacion_mas_65_anos,tamano_medio_hogar,prcnt_hogares_unipersonales,poblacion_habitantes,prcnt_poblacion_espanola
0,1700101001,17001,Agullana,2022,19392.0,50407.0,28845.0,19950.0,27842.0,72374.0,...,11688.0,30.8,2.7,44.0,19.1,20.6,2.6,26.3,921.0,83.8
1,1700101001,17001,Agullana,2021,15642.0,39854.0,23693.0,19250.0,20853.0,53129.0,...,5391.0,30.7,2.5,43.4,19.3,20.2,2.5,29.3,893.0,83.5
2,1700101001,17001,Agullana,2020,13498.0,34421.0,20669.0,17850.0,16650.0,42457.0,...,2098.0,30.8,2.6,43.6,18.1,21.2,2.5,29.4,888.0,82.6
3,1700101001,17001,Agullana,2019,14657.0,36909.0,21907.0,17850.0,18568.0,46757.0,...,3981.0,30.9,2.4,43.7,18.3,21.2,2.5,28.5,858.0,83.1
4,1700101001,17001,Agullana,2018,14209.0,35371.0,20930.0,17150.0,17936.0,44647.0,...,4181.0,31.8,2.4,43.9,18.1,21.8,2.5,31.6,830.0,84.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41179,2591301001,25913,Riu,2019,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41180,2591301001,25913,Riu,2018,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41181,2591301001,25913,Riu,2017,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41182,2591301001,25913,Riu,2016,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41184 entries, 0 to 41183
Data columns (total 24 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   codi_districte                                           41184 non-null  object 
 1   codi_postal                                              41184 non-null  object 
 2   poblacio                                                 41184 non-null  object 
 3   year                                                     41184 non-null  int64  
 4   renta_neta_mitja_por_persona                             40290 non-null  float64
 5   renta_neta_mitja_por_hogar                               40290 non-null  float64
 6   renta_mitja_por_unitat_consum                            40158 non-null  float64
 7   renta_mediana_por_unitat_consum                          40158 non-null  float64
 8   renta_bruta_mitja_por_pers

### Exportar

In [59]:
df.to_csv("dataset_ine.csv", index=False)
